# Example on analysing the dataset

In [14]:
import itertools as it

import joblib
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, execute, Aer

from utils import qc_reader

In [3]:
n_qubit = 4

In [35]:
assert n_qubit in [4, 8, 12, 16], "Incorrect number of qubits specified"

hamiltonian_names = [
    "1D transverse-field Ising model",
    "1D Heisenberg model",
    "Su-Schrieffer-Heeger model",
    "J1-J2 model",
    "1D Hubbard model",
    "2D Hubbard model",
]

ansatz_names = [
    "Hamiltonian",
    "Hardware-efficient",
    "Complete Hardware-efficient",
    "Ladder Hardware-efficient",
    "Cross-Ladder Hardware-efficient",
    "1D brick-block",
    "Stair brick-block",
    "Complete brick-block",
    "Ladder brick-block",
    "Cross-Ladder brick-block",
]

if n_qubit == 4:
    labels = list(range(5))
    hamiltonian_names = hamiltonian_names[:-1]
else:
    labels = list(range(6))

In [8]:
# load ground state
ground_state_list = []
for label in labels:
    ground_state = joblib.load(f"../data/ground_state/{str(n_qubit).zfill(2)}qubit/label{label}.jb")["ground_state"]
    ground_state_list.append(ground_state)

In [9]:
# load qasm_data
qasm_datas, hamiltonian_labels = qc_reader.load_qc(path="../data/qasm", n_qubit=n_qubit, label_kind="hamiltonian")
_, ansatz_labels = qc_reader.load_qc(path="../data/qasm", n_qubit=n_qubit, label_kind="ansatz")
_, reps_labels = qc_reader.load_qc(path="../data/qasm", n_qubit=n_qubit, label_kind="ansatz_reps")

In [10]:
def get_state_from_qasm(qasm_str: str):
    # convert qasm to QuantumCircuit object
    qc = QuantumCircuit.from_qasm_str(qasm_str)
    job = execute(qc, Aer.get_backend('statevector_simulator'))
    state = job.result().get_statevector()

    return state

In [17]:
rows = []
for i, qasm_str, hamiltonian_label, ansatz_label, reps_label in zip(list(range(len(qasm_datas))), qasm_datas,
                                                                    hamiltonian_labels, ansatz_labels, reps_labels):
    print(f"Converting: {(i + 1) / len(qasm_datas) * 100:.2f}%")
    state = get_state_from_qasm(qasm_str=qasm_str)
    fidelity = np.abs(np.vdot(state, ground_state_list[hamiltonian_label])) ** 2
    ansatz_reps = reps_label + 3

    row = [n_qubit, hamiltonian_names[hamiltonian_label], ansatz_names[ansatz_label], ansatz_reps, fidelity, state]
    rows.append(row)

Converting: 0.07%
Converting: 0.13%
Converting: 0.20%
Converting: 0.27%
Converting: 0.33%
Converting: 0.40%
Converting: 0.47%
Converting: 0.53%
Converting: 0.60%
Converting: 0.67%
Converting: 0.73%
Converting: 0.80%
Converting: 0.87%
Converting: 0.93%
Converting: 1.00%
Converting: 1.07%
Converting: 1.13%
Converting: 1.20%
Converting: 1.27%
Converting: 1.33%
Converting: 1.40%
Converting: 1.47%
Converting: 1.53%
Converting: 1.60%
Converting: 1.67%
Converting: 1.73%
Converting: 1.80%
Converting: 1.87%
Converting: 1.93%
Converting: 2.00%
Converting: 2.07%
Converting: 2.13%
Converting: 2.20%
Converting: 2.27%
Converting: 2.33%
Converting: 2.40%
Converting: 2.47%
Converting: 2.53%
Converting: 2.60%
Converting: 2.67%
Converting: 2.73%
Converting: 2.80%
Converting: 2.87%
Converting: 2.93%
Converting: 3.00%
Converting: 3.07%
Converting: 3.13%
Converting: 3.20%
Converting: 3.27%
Converting: 3.33%
Converting: 3.40%
Converting: 3.47%
Converting: 3.53%
Converting: 3.60%
Converting: 3.67%
Converting

## All data

In [18]:
col = ["qubit", "hamiltonian", "ansatz", "reps", "fidelity with ground state", "state"]
df = pd.DataFrame(rows, columns=col)
df

,qubit,hamiltonian,ansatz,reps,fidelity with ground state,state
0,4,1D transverse-field Ising model,Hamiltonian,3,1.0,"((0.010799235943719676-0.16852944722144272j), ..."
1,4,1D transverse-field Ising model,Hamiltonian,4,1.0,"((0.10026300279531955+0.1358906112183908j), (-..."
2,4,1D transverse-field Ising model,Hamiltonian,5,1.0,"((-0.11359210523192395+0.12496423439396644j), ..."
3,4,1D transverse-field Ising model,Hamiltonian,6,1.0,"((-0.167520358178403-0.02132770621866116j), (0..."
4,4,1D transverse-field Ising model,Hamiltonian,7,1.0,"((0.16758759327079617+0.020760810282311998j), ..."
...,...,...,...,...,...,...
1495,4,1D Hubbard model,Cross-Ladder brick-block,28,1.0,"((-1.6429562503371182e-09+0j), (3.917867114067..."
1496,4,1D Hubbard model,Cross-Ladder brick-block,29,1.0,"((1.9972267541837147e-08+0j), (-5.819322479450..."
1497,4,1D Hubbard model,Cross-Ladder brick-block,30,1.0,"((-6.160791634908235e-08+0j), (-3.762825245834..."
1498,4,1D Hubbard model,Cross-Ladder brick-block,31,1.0,"((1.5070955209037402e-07+0j), (-1.134420607451..."


## data grouped by ansatz

In [37]:
def get_fidelity_from_state_list(state_list) -> np.ndarray:
    """state_listから2つを選ぶ全ての組み合わせについて、Fidelityを求める"""
    fidelity_list = []
    for i, (state1, state2) in enumerate(it.combinations(state_list, 2)):
        fidelity = np.abs(np.vdot(state1, state2)) ** 2
        fidelity_list.append(fidelity)
    return np.array(fidelity_list)


def get_analysis_info(df: pd.DataFrame, hamiltonian_name, qubit, ansatz_name=None):
    if ansatz_name:
        _df = df.query(f"hamiltonian=='{hamiltonian_name}' & ansatz=='{ansatz_name}'")
    else:
        _df = df.query(f"hamiltonian=='{hamiltonian_name}'")

    if ansatz_name:
        result = [qubit, hamiltonian_name, ansatz_name]
    else:
        result = [qubit, hamiltonian_name]

    fidelity = _df["fidelity with ground state"].values
    fidelity_diff = get_fidelity_from_state_list(state_list=_df.state.values)
    result += [fidelity_diff.mean(), fidelity_diff.std(), fidelity_diff.max(),
               fidelity_diff.min(), fidelity.mean(), fidelity.std(), fidelity.max(),
               fidelity.min()]
    return result

In [43]:
rows = []
col = ["qubit", "hamiltonian", "ansatz", "mean of fidelity each data", "std", "max", "min",
       "mean of fidelity with ground state", "std", "max", "min"]

for hamiltonian_name in hamiltonian_names:
    for ansatz_name in ansatz_names:
        row = get_analysis_info(df=df.copy(), hamiltonian_name=hamiltonian_name, qubit=n_qubit, ansatz_name=ansatz_name)
        rows.append(row)
df2 = pd.DataFrame(rows, columns=col)
df2

,qubit,hamiltonian,ansatz,mean of fidelity each data,std,max,min,mean of fidelity with ground state,std,max,min
0,4,1D transverse-field Ising model,Hamiltonian,1.000000,1.871079e-09,1.000000,1.000000e+00,1.000000,1.383530e-09,1.000000,1.000000
1,4,1D transverse-field Ising model,Hardware-efficient,1.000000,5.826096e-12,1.000000,1.000000e+00,1.000000,4.135750e-12,1.000000,1.000000
2,4,1D transverse-field Ising model,Complete Hardware-efficient,1.000000,1.245515e-12,1.000000,1.000000e+00,1.000000,8.562325e-13,1.000000,1.000000
3,4,1D transverse-field Ising model,Ladder Hardware-efficient,1.000000,4.854841e-12,1.000000,1.000000e+00,0.998828,5.559192e-09,0.998829,0.998828
4,4,1D transverse-field Ising model,Cross-Ladder Hardware-efficient,1.000000,3.348762e-12,1.000000,1.000000e+00,1.000000,2.360841e-12,1.000000,1.000000
5,4,1D transverse-field Ising model,1D brick-block,1.000000,8.317790e-14,1.000000,1.000000e+00,1.000000,5.873805e-14,1.000000,1.000000
6,4,1D transverse-field Ising model,Stair brick-block,1.000000,8.546785e-14,1.000000,1.000000e+00,1.000000,6.102303e-14,1.000000,1.000000
7,4,1D transverse-field Ising model,Complete brick-block,1.000000,5.371589e-14,1.000000,1.000000e+00,1.000000,3.494923e-14,1.000000,1.000000
8,4,1D transverse-field Ising model,Ladder brick-block,1.000000,8.012512e-14,1.000000,1.000000e+00,1.000000,5.638688e-14,1.000000,1.000000
9,4,1D transverse-field Ising model,Cross-Ladder brick-block,1.000000,1.529571e-13,1.000000,1.000000e+00,1.000000,1.074726e-13,1.000000,1.000000


## data grouped by label

In [42]:
rows = []
col = ["qubit", "hamiltonian", "mean of fidelity each data", "std", "max", "min",
       "mean of fidelity with ground state", "std", "max", "min"]
for hamiltonian_name in hamiltonian_names:
    row = get_analysis_info(df=df.copy(), hamiltonian_name=hamiltonian_name, qubit=n_qubit)
    rows.append(row)
df3 = pd.DataFrame(rows, columns=col)
df3

,qubit,hamiltonian,mean of fidelity each data,std,max,min,mean of fidelity with ground state,std,max,min
0,4,1D transverse-field Ising model,0.999788,4.506716e-04,1.0,9.988269e-01,0.999883,3.514560e-04,1.0,0.998828
1,4,1D Heisenberg model,1.000000,2.181744e-12,1.0,1.000000e+00,1.000000,1.516597e-12,1.0,1.000000
2,4,Su-Schrieffer-Heeger model,0.903144,2.025903e-01,1.0,4.843456e-07,0.950000,1.500000e-01,1.0,0.500000
3,4,J1-J2 model,0.903216,2.024282e-01,1.0,1.857765e-06,0.950000,1.500000e-01,1.0,0.500000
4,4,1D Hubbard model,0.997304,5.742913e-03,1.0,9.850709e-01,0.998507,4.478625e-03,1.0,0.985071
